# POS tagger for German, NLP assignment 2, FHNW FS18

## Read and prepare data

In [1]:
import nltk
import numpy as np

In [2]:
# map Stuttgart-Tübingen tagset to universal tagset
stts_to_univ = nltk.tag.tagset_mapping('de-tiger', 'universal')

In [3]:
def readfile(filename):
    with open(filename, 'r') as f:
        lines = f.readlines() # sentences are split by newline
        
    tagged_sents = []
    for i, tagged_sent in enumerate(lines):
        tagged_sents.append([])
        for pair in tagged_sent.split(';'): # tagged words are split by ;
            splitted = pair.split('/') # word and tag are split by /
            if (len(splitted) == 2): # making sure we only get actual word-tag pairs
                word = splitted[0].strip().lower()
                tag = stts_to_univ[splitted[1].strip()] # map the stts tag to a universal tag
                tagged_sents[i].append((word, tag))
    
    return tagged_sents

In [4]:
tagged_sents = readfile("POS_German_train.txt")
tagged_sents_minitest = readfile("POS_German_minitest.txt")

In [5]:
from sklearn.model_selection import train_test_split

# split data so we have additional testdata (not only sentences_minitest)
tagged_sents_train, tagged_sents_test = train_test_split(tagged_sents, test_size=.1)

### Calculate emission probabilities P(word | tag)

In [6]:
#just here for completness, not needed anymore because nltk HMM trainer yields better results
import itertools
from nltk import FreqDist, ConditionalFreqDist
tagged_words = list(itertools.chain(*tagged_sents_train)) # flatten
emissions = ConditionalFreqDist((tag, word) for word, tag in tagged_words)

In [7]:
#example
print("Example: P(weit | ADJ) = " + str(emissions['ADJ'].freq('weit')))

Example: P(weit | ADJ) = 0.0024105461393596987


### Calculate transition probabilities P(tag_i | tag_j)

In [8]:
#just here for completness, not needed anymore because nltk HMM trainer yields better results
transitions = ConditionalFreqDist()
for tagged_sent in tagged_sents_train[:500]:
    transitions += ConditionalFreqDist([('START', tagged_sent[0][1])])
    for i in range(1, len(tagged_sent)):
        transitions += ConditionalFreqDist([(tagged_sent[i - 1][1], tagged_sent[i][1])])

In [9]:
#just here for completness, not needed anymore because nltk HMM trainer yields better results
# visualize transition probabilities
titlerow = []
titlerow.append('{0:5}'.format(''))
for tag in list(transitions):
    titlerow.append('{0:5}'.format(tag))

print(titlerow)
for tag1 in list(transitions):
    row = []
    row.append('{0:5}'.format(tag1))
    for tag2 in list(transitions):
        row.append('{:>.3f}'.format(transitions[tag1].freq(tag2)))
    print(row)

['     ', 'START', 'DET  ', 'NOUN ', 'ADV  ', 'VERB ', '.    ', 'PRON ', 'ADP  ', 'ADJ  ', 'PRT  ', 'CONJ ', 'NUM  ', 'X    ']
['START', '0.000', '0.178', '0.172', '0.096', '0.016', '0.042', '0.124', '0.172', '0.064', '0.010', '0.052', '0.024', '0.050']
['DET  ', '0.000', '0.000', '0.639', '0.009', '0.000', '0.024', '0.014', '0.018', '0.277', '0.003', '0.000', '0.011', '0.006']
['NOUN ', '0.000', '0.096', '0.106', '0.031', '0.226', '0.195', '0.017', '0.196', '0.038', '0.038', '0.042', '0.010', '0.004']
['ADV  ', '0.000', '0.106', '0.043', '0.127', '0.152', '0.073', '0.051', '0.195', '0.152', '0.033', '0.010', '0.046', '0.013']
['VERB ', '0.000', '0.109', '0.050', '0.052', '0.104', '0.395', '0.130', '0.066', '0.033', '0.012', '0.032', '0.005', '0.013']
['.    ', '0.000', '0.086', '0.158', '0.048', '0.171', '0.092', '0.134', '0.085', '0.065', '0.011', '0.131', '0.005', '0.015']
['PRON ', '0.000', '0.101', '0.287', '0.098', '0.140', '0.048', '0.074', '0.109', '0.098', '0.018', '0.007', '0

In [10]:
#just here for completness, not needed anymore because nltk HMM trainer yields better results
def viterbi(U,E,p0,o):
    k = o.size
    (n,m) = E.shape
    theta = np.zeros( (n,k) )
    psi = np.zeros( (n,k) )
    theta[:, 0] = p0*E[:,int(o[0,0])]
    psi[:,0] = np.arange(n)
    for obs_ind in range(1,k,1):
        theta_old = theta[:,obs_ind-1]
        for i in range(0,n,1):
            temp = U[:,i]*theta_old
            theta[i, obs_ind] = E[i,int(o[0,obs_ind])]*max(temp)
            psi[i, obs_ind] = np.argmax(temp)
    p = max(theta[:,k-1])
    q = np.zeros(k)
    q[obs_ind] = np.argmax(theta[:,k-1])
    for obs_ind in range(k-1,0,-1):
        q[obs_ind-1]=psi[int(q[obs_ind]),obs_ind]
    return p,q

def forward_backward(U,E,p0,o):
    k = o.size
    (n,m) = E.shape
    fw = np.zeros( (n,k) )
    bw = np.zeros( (n,k) )
    fw[:, 0] = p0*E[:,int(o[0,0])]
    bw[:, k-1] = np.ones((n,1))[:,0]
    for obs_ind in range(1,k,1):
        fw_old = np.transpose(fw[:,obs_ind-1])
        fw[:, obs_ind] = fw_old.dot(U)*E[:,int(o[0,obs_ind])]
        bw_old = bw[:,k-obs_ind]
        bw[:, k-obs_ind-1] = U.dot(bw_old*E[:,int(o[0,k-obs_ind])])
    p = np.sum(fw[:,k-1])    
    return p,fw,bw

## Set up some taggers to determine POS

In [11]:
from nltk import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

default_tagger = DefaultTagger('NOUN')
patterns = [
    (r'^‐?[0‐9]*(.[0‐9]+)?$', 'NUM'),
    (r'^[0‐9]*-[0‐9]*$', 'NUM'),
    (r'`ge.*tet$',   'VERB')
]
regex_tagger = nltk.RegexpTagger(patterns ,backoff=default_tagger)
unigram_tagger = UnigramTagger(tagged_sents_train, backoff=regex_tagger)
bigram_tagger = BigramTagger(tagged_sents_train, backoff=unigram_tagger)
trigram_tagger = TrigramTagger(tagged_sents_train, backoff=bigram_tagger)

In [12]:
from nltk.tag.hmm import HiddenMarkovModelTrainer, HiddenMarkovModelTagger
from nltk.probability import LidstoneProbDist, UniformProbDist

tag_set = set([tag for sentence in tagged_sents for word, tag in sentence])
word_set = set([word for sentence in tagged_sents for word, tag in sentence])

#hmm_tagger = HiddenMarkovModelTagger(word_set, tag_set, transitions, emissions, UniformProbDist(tag_set))
#using the nltk HMM Trainer yields better results!
hmm_trainer = HiddenMarkovModelTrainer(list(tag_set), list(word_set)) #make sets to lists again so items can potentionally be appended
hmm_tagger = hmm_trainer.train_supervised(tagged_sents_train, estimator=lambda fd, bins: LidstoneProbDist(fd, .1, bins))

In [13]:
#testing some random example
trigram_tagger.tag("Der schnelle Fuchs springt über den faulen Zaun".split())

[('Der', 'NOUN'),
 ('schnelle', 'ADJ'),
 ('Fuchs', 'NOUN'),
 ('springt', 'VERB'),
 ('über', 'NOUN'),
 ('den', 'DET'),
 ('faulen', 'NOUN'),
 ('Zaun', 'NOUN')]

In [14]:
hmm_tagger.tag("Der schnelle Fuchs springt über den faulen Zaun".split())

[('Der', 'DET'),
 ('schnelle', 'ADJ'),
 ('Fuchs', 'NOUN'),
 ('springt', 'VERB'),
 ('über', 'ADP'),
 ('den', 'DET'),
 ('faulen', 'ADJ'),
 ('Zaun', 'NOUN')]

In [15]:
# Evaluate all the taggers and using a confusion matrix to compare predicted vs actual tags
testset = tagged_sents_minitest #+ tagged_sents_test
taggers = [trigram_tagger, hmm_tagger]

for tagger in taggers:
    predicted_tags = [tag for tagged_sent in testset
                      for (word, tag) in tagger.tag(
                          list(map(lambda tagged_word: tagged_word[0], tagged_sent)))]
    actual_tags = [tag for tagged_sent in testset
                   for tag in list(map(lambda tagged_word: tagged_word[1], tagged_sent))]
    print(tagger)
    print(tagger.evaluate(testset))
    print(nltk.ConfusionMatrix(predicted_tags, actual_tags))

<TrigramTagger: size=3971>
0.9487179487179487
     |                        C         N         P         V      |
     |         A    A    A    O    D    O    N    R    P    E      |
     |         D    D    D    N    E    U    U    O    R    R      |
     |    .    J    P    V    J    T    N    M    N    T    B    X |
-----+-------------------------------------------------------------+
   . |<1167>   .    .    .    .    .    .    .    .    .    .    . |
 ADJ |    . <552>   .    2    .    .   11    .    4    .    8    . |
 ADP |    .    .<1000>  15    8    .    .    .    .   29    .    . |
 ADV |    .   16    4 <337>   4    .    1    .    4    1    .    . |
CONJ |    .    .    4    5 <294>   .    3    .    .    .    .    1 |
 DET |    .    .    .    .    . <898>   1    2   18    1    .    . |
NOUN |    .  123    .    4    .    .<2366>   9    1    1   63   11 |
 NUM |    .    .    .    .    .    .    . <153>   .    .    .    . |
PRON |    .    1    .    6    2   25    .    . <419>   . 

## Conclusion
For the given testset, the trigram tagger seems to yield better results than the HMM tagger (~95% vs ~93% accuracy).
Also the HMM tagger is a lot slower than the stacked trigram-tagger.

In [16]:
# print a list of the wrong predictions for analisys
tagged_words = list(itertools.chain(*testset)) # flatten
for i in range(len(predicted_tags)):
    if (predicted_tags[i] != actual_tags[i]):
        print(f'predicted: {predicted_tags[i]}\t,actual:{actual_tags[i]}\t({tagged_words[i][0]})\n')

predicted: PRON	,actual:NOUN	(expertenmeinung)

predicted: X	,actual:NOUN	(nuklearanlagen)

predicted: PRON	,actual:NOUN	(chemiewerken)

predicted: ADP	,actual:ADJ	(tausender)

predicted: VERB	,actual:NOUN	(zechner)

predicted: .	,actual:VERB	(nachgezogen)

predicted: VERB	,actual:NOUN	(beinhofer)

predicted: X	,actual:NOUN	(beinhofer)

predicted: VERB	,actual:NOUN	(schenkelberg)

predicted: PRON	,actual:NOUN	(city-streifen)

predicted: ADV	,actual:ADP	(bis)

predicted: ADJ	,actual:NOUN	(formel-1-veranstalter)

predicted: PRT	,actual:NOUN	(ecclestone)

predicted: PRT	,actual:NOUN	(boersenplaetzen)

predicted: ADP	,actual:ADV	(zu)

predicted: CONJ	,actual:VERB	(schaetzten)

predicted: .	,actual:NOUN	(ecclestone)

predicted: PRT	,actual:ADJ	(aktiver)

predicted: CONJ	,actual:ADP	(ohne)

predicted: ADP	,actual:CONJ	(als)

predicted: DET	,actual:NOUN	(itv)

predicted: ADJ	,actual:VERB	(bezahlte)

predicted: NOUN	,actual:X	(grand)

predicted: NOUN	,actual:X	(prix)

predicted: DET	,actual:NO